In [1]:
%matplotlib inline
from scipy.stats import ttest_ind

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import scanpy as sc
import numpy as np

from sctools import score


mpl.rcParams['pdf.fonttype'] = 42
np.random.seed(1902834)

Global seed set to 0


In [2]:
genes = pd.read_csv(
    '../resource/fragility_score_genes.txt',
    sep = '\t'
)
genes

,gene,type
0,CCL4,cytokine_chemokine
1,CCL5,cytokine_chemokine
2,CSF2,cytokine_chemokine
3,IFNG,cytokine_chemokine
4,IL13,cytokine_chemokine
5,IL17A,cytokine_chemokine
6,IL2,cytokine_chemokine
7,IL22,cytokine_chemokine
8,IL6,cytokine_chemokine
9,IL9,cytokine_chemokine


In [3]:
adata = sc.read_h5ad(
    '../data/tregs.tissue.scps.integrated.annotated.h5ad'
)
adata

AnnData object with n_obs × n_vars = 9773 × 20912
    obs: 'sample_id', 'patient_id', 'status', 'tissue', 'cell_fraction', 'doublet', 'doublet_score', 'nFeature_RNA', 'nCount_RNA', 'percent_mt', 'percent_ribo', 'qc_pass', '_scvi_batch', '_scvi_labels', 'leiden_scvi_0.1', 'coarse_cell_types', 'leiden_scvi_0.4', 'dataset', 'sat1_status', 'clustering', 'sat1_status_majority_vote'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'leiden', 'log1p', 'neighbors', 'umap'
    obsm: 'X_scvi', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [4]:
adata.X = adata.layers['counts']
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)

In [5]:
score_df = pd.DataFrame(
    {
        'fragility_score': score.gene_module_score(adata, genes.gene),
        'sat1_status': adata.obs.sat1_status_majority_vote,
        'sat1_expression': adata[:, 'SAT1'].X.toarray().flatten()
    },
    index = adata.obs.index
)

fig, ax = plt.subplots()
sns.violinplot(
    data = score_df,
    x = 'sat1_status',
    y = 'fragility_score',
    ax = ax
)
# sns.stripplot(
#     data = score_df,
#     x = 'sat1_status',
#     y = 'fragility_score',
#     ax = ax,
#     c = 'k',
#     alpha = 0.25
# )
ttest_results = ttest_ind(
    score_df.loc[score_df.sat1_status == 'SAT1_hi', 'fragility_score'],  
    score_df.loc[score_df.sat1_status == 'SAT1_lo', 'fragility_score']
)
_, ymax = ax.get_ylim()
xmin, _ = ax.get_xlim()
ax.text(
    xmin, ymax,
    str(ttest_results.pvalue),
    va = 'top'
)
fig.set_figheight(5)
fig.set_figwidth(3)
fig.tight_layout()
fig.savefig('../plots/tregs.tissue.scps.sat1.fragility.pdf')